<a href="https://colab.research.google.com/github/ShotaSasaki-HCU/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Pandas
# output.clear() # 出力画面をクリアする。

import pandas as pd
from google.colab import output # 出力画面クリア用
import datetime
import numpy as np

# header=0として、csvの１行目は列名と指定する。
df = pd.read_csv('https://raw.githubusercontent.com/ShotaSasaki-HCU/Prog2kakushin/main/netflix%20daily%20top%2010.csv', header=0)

separate = '-----------------------------------------------------------' # 区切り横線
functions = ["トップ10入り合計日数ランキング", "最大連続トップ10入り日数ランキング"] # 機能一覧を格納するリストfunctions
error_enable = False # 選択肢範囲外をユーザが選択したとき用のフラグ
while True:
  output.clear() # 選択肢範囲外を選択し、戻ってきた場合のため出力画面をクリアする。
  print("【2020年4月1日〜2022年3月11日のNetflixランキング分析】")
  for i, function in enumerate(functions): # リストfunctionsを箇条書きで列挙する。
    print(str(i+1) + '.', function)
  if error_enable:
    print("[エラー]1~" + str(len(functions)) + "の範囲で入力してください。")
  select = int(input(separate + "\n選択肢の番号を入力："))
  if not(1 <= select <= len(functions)): # 選択肢範囲外をユーザが指定したらwhile文の先頭まで戻る。
    error_enable = True
    continue
  error_enable = False

  # 1. トップ10入り合計日数ランキング
  if select == 1:
    df_title = df['Title'].value_counts() # ユニークな要素のそれぞれの出現回数を得る。
    while True:
      output.clear() # 画面遷移のため、出力画面をクリアする。
      print("【1. トップ10入り合計日数ランキング】")
      if error_enable:
        print("[エラー]1~" + str(len(df_title)) + "の範囲で入力してください。")
      limit = int(input("上位何位(1~" + str(len(df_title)) + ")まで表示しますか？："))
      if not(1 <= limit <= len(df_title)): # 選択肢範囲外をユーザが指定したらwhile文の先頭まで戻る。
        error_enable = True
        continue
      error_enable = False
      break
    titles = df_title[0:limit].index.values # 出現回数が格納されたSeriesの行名（作品タイトル）をリスト化
    print('   ' + 'Title'.center(30) + ':\t出現日数（／' + str(int(len(df)/10)) + '日）\n' + separate)
    for i in range(len(titles)):
      print(str(i+1) + '. ' + titles[i].center(30) + ':\t' + str(df_title[0:limit][i]).center(16))

  # 2. 最大連続トップ10入り日数ランキング
  elif select == 2:
    titles = list(df['Title'].unique()) # 作品タイトルの列を重複無しで抽出してリスト化する。
    in_a_row_temp = dict(zip(titles, [0]*len(titles))) # 暫定的な連続日数
    in_a_row_final = dict(zip(titles, [0]*len(titles))) # 最大連続日数

    top10_yesterday = list(df['Title'][0:10]) # 前の日付の10タイトルを格納するリスト
    for i in range(10, int(len(df)/10), 10): # 1日ずつ見ていく。
      top10_today = list(df['Title'][i:i+10]) # 現在の日付の10タイトルをリストにする。
      for title_yesterday in top10_yesterday: # 前の日付のタイトルが、
        if title_yesterday in top10_today: # 現在の日付の10タイトルに残っていたら、
          in_a_row_temp[title_yesterday] += 1 # それを連続として加算する。
        else: # 前の日付のタイトルが、現在の日付の10タイトルに残っていなかったら、
          if in_a_row_temp[title_yesterday] > in_a_row_final[title_yesterday]: # 暫定的な連続日数が、今までの連続日数を超えていたら、
            in_a_row_final[title_yesterday] = in_a_row_temp[title_yesterday] # 最大連続日数を、暫定的だった連続日数で書き換える。
          in_a_row_temp[title_yesterday] = 0 # 連続が途切れたので、暫定的な連続日数をリセットする。
      top10_yesterday = top10_today # 現在の日付の処理が終わったら、現在のタイトルを前のタイトルとして記録する。
    # print(in_a_row_final)

    title_c = df['Title'].value_counts() # ユニークな要素のそれぞれの出現回数を得る。
    titles = title_c[0:].index.values
    for j in range(0, len(titles)):
      print(titles[j] + '\t' + str(title_c[j]) + '\t' + str(in_a_row_final[titles[j]])) # あれ？ココメロンは、例えば2020/8/28と8/29で連続してランクインしているのにカウント0になっている。

  break

【2020年4月1日〜2022年3月11日のNetflixランキング分析】
1. トップ10入り合計日数ランキング
2. 最大連続トップ10入り日数ランキング
-----------------------------------------------------------
選択肢の番号を入力：2
Cocomelon	428	0
Ozark	85	31
Cobra Kai	81	0
Manifest	80	0
The Queenâs Gambit	73	0
Outer Banks	72	38
Squid Game	66	0
All American	58	17
Bridgerton	58	0
Lucifer	56	0
Virgin River	55	0
Maid	49	0
Emily in Paris	48	0
Too Hot to Handle	47	13
Sweet Magnolias	45	18
The Witcher	45	0
The Queen of Flow	44	0
Ginny & Georgia	44	0
Who Killed Sara?	42	0
Shameless	42	0
iCarly	42	0
The Circle US	40	0
Money Heist	39	15
Avatar: The Last Airbender	39	0
Love is Blind	38	0
You	35	0
The Umbrella Academy	34	0
Riverdale	34	21
Clickbait	34	0
Unsolved Mysteries	33	0
Space Force	32	0
The Mitchells vs. The Machines	31	0
Jeffrey Epstein: Filthy Rich	30	0
Tiger King: Murder, Mayhem …	29	26
Vivo	29	0
All of Us Are Dead	29	0
How the Grinch Stole Christmas	29	0
Never Have I Ever	28	12
365 Days	28	0
Inventing Anna	28	0
Firefly Lane	27	0
Despicable Me 2	27	0
Sex/Life	26	